# Atividade Ponderada: Otimização de Modelo Pré-Treinado para Detecção de Fraudes em Cartões de Crédito
## Aluno: Henrique Cox

## Importando o database

In [ ]:
%pip install gdown
import gdown

In [ ]:
arquivo_destino_colab = "dataset.csv"
doc_id = "1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl"
URL = f"https://drive.google.com/uc?id={doc_id}"
gdown.download(URL, arquivo_destino_colab, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl
From (redirected): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl&confirm=t&uuid=7621e21c-02d4-4782-a01f-c5a7b11c8b16
To: /content/dataset.csv
100%|██████████| 151M/151M [00:01<00:00, 78.1MB/s]


'dataset.csv'

## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

## Pré Processamento

In [ ]:
df = pd.read_csv("dataset.csv")

display(df.head(10))

print("Colunas:", list(df.columns))
print("Formato (linhas, colunas):", df.shape)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


Colunas: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
Formato (linhas, colunas): (284807, 31)


In [ ]:
na = df.isna().sum()
print("Valores ausentes por coluna (se houver):")
print(na[na > 0] if na.sum() > 0 else "Nenhum NA encontrado.")

print("\nDistribuição da classe:")
class_counts = df["Class"].value_counts().sort_index()
print(class_counts)
print("Proporções:")
print((class_counts / len(df)).round(4))

print("\nEstatísticas de Time e Amount:")
print(df[["Time", "Amount"]].describe())

dup_count = df.duplicated().sum()
print(f"\nDuplicatas exatas: {dup_count}")


Valores ausentes por coluna (se houver):
Nenhum NA encontrado.

Distribuição da classe:
Class
0    284315
1       492
Name: count, dtype: int64
Proporções:
Class
0    0.9983
1    0.0017
Name: count, dtype: float64

Estatísticas de Time e Amount:
                Time         Amount
count  284807.000000  284807.000000
mean    94813.859575      88.349619
std     47488.145955     250.120109
min         0.000000       0.000000
25%     54201.500000       5.600000
50%     84692.000000      22.000000
75%    139320.500000      77.165000
max    172792.000000   25691.160000

Duplicatas exatas: 1081


In [ ]:
df_clean = df.drop_duplicates().copy()
removidas = len(df) - len(df_clean)
print(f"Duplicatas removidas: {removidas}")
print("Novo formato:", df_clean.shape)

X = df_clean.drop(columns=["Class"])
y = df_clean["Class"].astype("int8")

print("\nPrimeiras colunas de X:", list(X.columns)[:8], "...")
print("Total de features:", X.shape[1])
print("Total de linhas (após limpeza):", X.shape[0])

Duplicatas removidas: 1081
Novo formato: (283726, 31)

Primeiras colunas de X: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7'] ...
Total de features: 30
Total de linhas (após limpeza): 283726


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Tamanhos:")
print("Treino:", X_train.shape, "Teste:", X_test.shape)

scaler = StandardScaler()
X_train[["Time", "Amount"]] = scaler.fit_transform(X_train[["Time", "Amount"]])
X_test[["Time", "Amount"]] = scaler.transform(X_test[["Time", "Amount"]])

print("\nExemplo depois do scaling:")
print(X_train[["Time", "Amount"]].head())


Tamanhos:
Treino: (226980, 30) Teste: (56746, 30)

Exemplo depois do scaling:
            Time    Amount
226238  1.045499 -0.229434
134253 -0.298690 -0.331197
186465  0.678397 -0.298809
149493 -0.074929 -0.289247
18461  -1.376728 -0.261985


### Resumo

Na primeira parte, eu comecei carregando o dataset (dataset.csv) e dando um head pra visualizar as primeiras linhas e entender a estrutura. Logo de cara vi que ele tinha 31 colunas e mais de 280 mil linhas, sendo que as features iam de V1 até V28, além de Time, Amount e a coluna alvo Class.

Depois, fui verificar a qualidade do dataset. Confirmei que não havia valores nulos, o que facilita bastante o trabalho, mas percebi duas coisas:
1. O conjunto era muito desbalanceado, a classe 0 (não fraude) representava 99,8% dos registros, enquanto a classe 1 (fraude) era só 0,17%. Isso já mostra que qualquer modelo pode ter uma accuracy alta sem realmente identificar as fraudes, então métricas como recall e AUC-ROC vão ser bem mais relevantes.

2. Havia 1081 duplicatas. Como não fazia sentido manter registros idênticos, removi todas. Com isso, o dataset ficou com 283.726 linhas.

Em seguida, separei as variáveis em X (features) e y (target Class). Nesse ponto, X ficou com 30 colunas e y com os rótulos 0 e 1.

O próximo passo foi fazer o train/test split com 80% para treino e 20% para teste, usando stratify=y.

Por último, fiz a padronização (scaling). Como os atributos V1–V28 já são resultados de uma transformação (PCA), não mexi neles. Apliquei o StandardScaler apenas em Time e Amount, que estavam em escalas bem diferentes. Depois do scaling, eles ficaram com média 0 e desvio padrão de 1, o que ajuda a rede neural a treinar.


## Modelo Baseline

In [ ]:
model = Sequential([
    Dense(16, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(8, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=2048,
    verbose=1
)

y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

print("\n--- Relatório de Classificação ---")
print(classification_report(y_test, y_pred, digits=4))

auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC: {auc:.4f}")


Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9044 - loss: 0.3604 - val_accuracy: 0.9984 - val_loss: 0.0567
Epoch 2/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9987 - loss: 0.0389 - val_accuracy: 0.9989 - val_loss: 0.0161
Epoch 3/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9990 - loss: 0.0124 - val_accuracy: 0.9991 - val_loss: 0.0096
Epoch 4/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9992 - loss: 0.0077 - val_accuracy: 0.9991 - val_loss: 0.0074
Epoch 5/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9993 - loss: 0.0056 - val_accuracy: 0.9991 - val_loss: 0.0064
1774/1774 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step

--- Relatório de Classificação ---
              precision    recall  f1-score   support

           0     0.9992    0.9998    0.9995     56651
           1     0.8525    0.5474    0.6667        95

    accuracy                         0.9991     56746
   macro avg     0.9259    0.7736    0.8331     56746
weighted avg     0.9990 

### Resumo

A ideia deste modelo baseline era ter uma referência de desempenho, antes de aplicar qualquer outra otimização.

O modelo é simples: uma camada com 16 neurônios, outra com 8, ambas com ativação ReLU, e uma saída com 1 neurônio usando sigmoid (já que o problema é binário).

O treino foi feito por apenas 5 épocas e com um batch grande (2048), justamente para ser rápido e não exagerar na complexidade para esse modelo.

Nos resultados, a accuracy chegou em 99,9%, mas como esperado isso não significa tanto, porque a maioria dos dados são da classe 0 (não fraude). O que realmente importa são as métricas na classe 1. O modelo conseguiu precision de 0,85 e recall de 0,55 para fraudes, com F1 de 0,67. Isso mostra que, embora quando ele classifica algo como fraude geralmente esteja certo, ele ainda deixa passar quase metade dos casos de fraude.

Por fim, o AUC-ROC ficou em 0,90, o que é um valor bom e mostra que o modelo tem certa capacidade de separar as duas classes.

## Random Search

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

def build_model(input_dim, learning_rate=1e-3, units=16, n_hidden=2):
    model = Sequential()
    model.add(Dense(units, activation="relu", input_shape=(input_dim,)))
    for _ in range(max(0, n_hidden - 1)):
        model.add(Dense(max(4, units // 2), activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

def make_class_weight(y):
    n = len(y)
    n_pos = int((y == 1).sum())
    n_neg = n - n_pos
    return {0: n/(2*n_neg), 1: n/(2*n_pos)}

rng = np.random.default_rng(42)
param_space = {
    "learning_rate": [1e-2, 1e-3, 3e-4],
    "units": [8, 16, 32],
    "n_hidden": [1, 2],
    "batch_size": [1024, 2048],
    "epochs": [6, 8, 10],
    "use_class_weight": [False, True],
}

def sample_params(space, n_samples=8, rng=rng):
    keys = list(space.keys())
    samples = []
    for _ in range(n_samples):
        p = {k: rng.choice(space[k]) for k in keys}
        samples.append(p)
    return samples

candidates = sample_params(param_space, n_samples=8, rng=rng)

best_auc = -1.0
best_params = None
es = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True, verbose=0)

print("== Random Search (8 combinações) ==")
for i, p in enumerate(candidates, 1):
    model = build_model(
        input_dim=X_tr.shape[1],
        learning_rate=p["learning_rate"],
        units=p["units"],
        n_hidden=p["n_hidden"],
    )
    cw = make_class_weight(y_tr) if p["use_class_weight"] else None

    model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        epochs=p["epochs"],
        batch_size=p["batch_size"],
        verbose=0,
        callbacks=[es],
        class_weight=cw
    )

    y_val_prob = model.predict(X_val, verbose=0).ravel()
    auc = roc_auc_score(y_val, y_val_prob)
    print(f"[{i:02d}] AUC-VAL={auc:.4f} | {p}")

    if auc > best_auc:
        best_auc = auc
        best_params = p

print("\n>> Melhor combinação (Random Search):")
print(best_params, f"| AUC-VAL={best_auc:.4f}")

print("\n### Avaliação final no TESTE ###")
cw_full = make_class_weight(y_train) if best_params["use_class_weight"] else None
best_model = build_model(
    input_dim=X_train.shape[1],
    learning_rate=best_params["learning_rate"],
    units=best_params["units"],
    n_hidden=best_params["n_hidden"],
)

best_model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=best_params["epochs"],
    batch_size=best_params["batch_size"],
    verbose=0,
    callbacks=[es],
    class_weight=cw_full
)

y_test_prob = best_model.predict(X_test, verbose=0).ravel()
y_test_pred = (y_test_prob > 0.5).astype(int)

print("--- Métricas no TESTE ---")
print(classification_report(y_test, y_test_pred, digits=4))
print("AUC-ROC (teste):", roc_auc_score(y_test, y_test_prob))

== Random Search (8 combinações) ==


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[01] AUC-VAL=0.9713 | {'learning_rate': np.float64(0.01), 'units': np.int64(32), 'n_hidden': np.int64(2), 'batch_size': np.int64(1024), 'epochs': np.int64(8), 'use_class_weight': np.True_}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[02] AUC-VAL=0.9771 | {'learning_rate': np.float64(0.01), 'units': np.int64(32), 'n_hidden': np.int64(1), 'batch_size': np.int64(1024), 'epochs': np.int64(8), 'use_class_weight': np.True_}


...

[07] AUC-VAL=0.9777 | {'learning_rate': np.float64(0.0003), 'units': np.int64(32), 'n_hidden': np.int64(1), 'batch_size': np.int64(2048), 'epochs': np.int64(6), 'use_class_weight': np.True_}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[08] AUC-VAL=0.3259 | {'learning_rate': np.float64(0.0003), 'units': np.int64(16), 'n_hidden': np.int64(1), 'batch_size': np.int64(2048), 'epochs': np.int64(8), 'use_class_weight': np.True_}

>> Melhor combinação (Random Search):
{'learning_rate': np.float64(0.0003), 'units': np.int64(32), 'n_hidden': np.int64(1), 'batch_size': np.int64(2048), 'epochs': np.int64(6), 'use_class_weight': np.True_} | AUC-VAL=0.9777

### Avaliação final no TESTE ###


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


--- Métricas no TESTE ---
              precision    recall  f1-score   support

           0     0.9997    0.9670    0.9831     56651
           1     0.0415    0.8526    0.0791        95

    accuracy                         0.9668     56746
   macro avg     0.5206    0.9098    0.5311     56746
weighted avg     0.9981    0.9668    0.9816     56746

AUC-ROC (teste): 0.9361953939587631


### Resumo

A próxima etapa foi aplicar uma técnica de random search para tentar encontrar combinações de hiperparâmetros que melhorassem o desempenho.

O random search testou 8 combinações aleatórias e escolhi a melhor com base no AUC-ROC da validação. O resultado foi uma configuração com learning rate baixo (0.0003), 32 neurônios, 1 camada escondida, batch 2048, 6 épocas e uso de class weights.

Nos resultados finais, o recall da classe fraude saltou para 0.85, ou seja, o modelo conseguiu detectar a grande maioria das fraudes. Porém, isso veio com um problema: o precision caiu para 0.04, fazendo o F1 cair para 0.08. Na prática, o modelo passou a classificar muitas transações normais como fraude (falsos positivos), mas quase não deixou fraude passar.

## Grid Search

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

def build_model(input_dim, learning_rate=1e-3, units=16, n_hidden=2):
    model = Sequential()
    model.add(Dense(units, activation="relu", input_shape=(input_dim,)))
    for _ in range(max(0, n_hidden - 1)):
        model.add(Dense(max(4, units // 2), activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

def make_class_weight(y):
    n = len(y)
    n_pos = int((y == 1).sum())
    n_neg = n - n_pos
    return {0: n/(2*n_neg), 1: n/(2*n_pos)}

grid = {
    "learning_rate": [1e-3, 3e-4],
    "units": [16, 32],
    "n_hidden": [1, 2],
    "batch_size": [1024],
    "epochs": [8, 10],
    "use_class_weight": [False, True],
}

import itertools
keys = list(grid.keys())
combos = list(itertools.product(*[grid[k] for k in keys]))
print(f"Total de combinações no grid: {len(combos)}")

best_auc = -1.0
best_params = None
es = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True, verbose=0)

for i, values in enumerate(combos, 1):
    params = dict(zip(keys, values))
    model = build_model(
        input_dim=X_tr.shape[1],
        learning_rate=params["learning_rate"],
        units=params["units"],
        n_hidden=params["n_hidden"],
    )
    cw = make_class_weight(y_tr) if params["use_class_weight"] else None

    model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        epochs=params["epochs"],
        batch_size=params["batch_size"],
        verbose=0,
        callbacks=[es],
        class_weight=cw
    )

    y_val_prob = model.predict(X_val, verbose=0).ravel()
    auc_val = roc_auc_score(y_val, y_val_prob)
    print(f"[{i:02d}] AUC-VAL={auc_val:.4f} | {params}")

    if auc_val > best_auc:
        best_auc = auc_val
        best_params = params

print("\n>> Melhor combinação (Grid Search):")
print(best_params, f"| AUC-VAL={best_auc:.4f}")

print("\n### Avaliação final no TESTE (melhor Grid) ###")
cw_full = make_class_weight(y_train) if best_params["use_class_weight"] else None
best_model = build_model(
    input_dim=X_train.shape[1],
    learning_rate=best_params["learning_rate"],
    units=best_params["units"],
    n_hidden=best_params["n_hidden"],
)

best_model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=best_params["epochs"],
    batch_size=best_params["batch_size"],
    verbose=0,
    callbacks=[es],
    class_weight=cw_full
)

y_test_prob = best_model.predict(X_test, verbose=0).ravel()
y_test_pred = (y_test_prob > 0.5).astype(int)

print("--- Métricas no TESTE (Grid) ---")
print(classification_report(y_test, y_test_pred, digits=4))
print("AUC-ROC (teste):", roc_auc_score(y_test, y_test_prob))

Total de combinações no grid: 32


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[01] AUC-VAL=0.9691 | {'learning_rate': 0.001, 'units': 16, 'n_hidden': 1, 'batch_size': 1024, 'epochs': 8, 'use_class_weight': False}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[02] AUC-VAL=0.9756 | {'learning_rate': 0.001, 'units': 16, 'n_hidden': 1, 'batch_size': 1024, 'epochs': 8, 'use_class_weight': True}


...

[31] AUC-VAL=0.9582 | {'learning_rate': 0.0003, 'units': 32, 'n_hidden': 2, 'batch_size': 1024, 'epochs': 10, 'use_class_weight': False}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[32] AUC-VAL=0.9757 | {'learning_rate': 0.0003, 'units': 32, 'n_hidden': 2, 'batch_size': 1024, 'epochs': 10, 'use_class_weight': True}

>> Melhor combinação (Grid Search):
{'learning_rate': 0.001, 'units': 32, 'n_hidden': 2, 'batch_size': 1024, 'epochs': 10, 'use_class_weight': False} | AUC-VAL=0.9826

### Avaliação final no TESTE (melhor Grid) ###


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


--- Métricas no TESTE (Grid) ---
              precision    recall  f1-score   support

           0     0.9995    0.9998    0.9997     56651
           1     0.8625    0.7263    0.7886        95

    accuracy                         0.9993     56746
   macro avg     0.9310    0.8631    0.8941     56746
weighted avg     0.9993    0.9993    0.9993     56746

AUC-ROC (teste): 0.9549026402655595


Depois do random search, eu rodei um grid search curto pra refinar os hiperparâmetros em torno do que já tinha funcionado.

O melhor conjunto de hiperparâmetros no grid saiu como:
learning_rate=0.001, units=32, n_hidden=2, batch_size=1024, epochs=10, use_class_weight=False
com AUC-VAL = 0.9826.

Depois, treinei o modelo com esses hiperparâmetros no treino completo (com validation_split=0.1 só pra monitorar o EarlyStopping) e avaliei no conjunto de teste.

Comparado ao random search, o grid search achou um equilíbrio bem melhor, já que ele manteve um recall alto (0.73) sem derrubar a precisão, na verdade, a precisão ficou alta (0.86). Isso elevou o F1 para 0.79 e ainda entregou o maior AUC-ROC entre os modelos que testei. Na prática, esse modelo erra menos por “alarme falso” e ainda detecta a maior parte das fraudes, então faz mais sentido como escolha final.

## Relatório Final

#### Comparação Final:

**Baseline**

- Recall (fraudes): 0.55

- Precision (fraudes): 0.85

- F1 (fraudes): 0.67

- AUC-ROC: 0.90

Modelo equilibrado, mas perde quase metade das fraudes.

---

**Random Search**

- Recall: 0.85

- Precision: 0.04

- F1: 0.08

- AUC-ROC: 0.94

Detecta quase todas as fraudes, mas com precisão baixa (muitos falsos positivos). É um modelo mais “paranoico”, bom para cenários onde não pode deixar a fraude escapar, mesmo que custe revisar alertas falsos.

---

**Grid Search**

- Recall (fraudes): 0.73

- Precision (fraudes): 0.86

- F1 (fraudes): 0.79

- AUC-ROC: 0.95

Aqui conseguimos o melhor equilíbrio: recall alto (detecta boa parte das fraudes), precision alto (a maioria dos alertas realmente são fraudes) e maior AUC-ROC de todos (0.95).

---

**Conclusão**

O baseline já apresentava bom desempenho, mas deixava passar muitas fraudes.

O random search maximizou o recall, quase não deixando fraudes passar, mas com custo de muitos falsos positivos.

O grid search conseguiu equilibrar recall e precision, entregando o melhor F1 e melhor AUC-ROC entre todos.

Assim, o grid search mostrou que o ajuste de hiperparâmetros é capaz de melhorar tanto a detecção de fraudes quanto a qualidade dos alertas, e provou ser a melhor configuração final, dentre as opções.